In [7]:
import os

In [14]:
files = os.listdir("catalogs/3sigma")

In [10]:
def create_transient_candidate_catalog(directory, filename, d_directory):
    
    import lsst.afw.table as afwTable

    
    catalog = afwTable.SourceCatalog.readFits(directory + "/" + filename)
    c_r  = catalog.copy()
    c_r.clear()
    for source in catalog:
        if source["classification_dipole"] == 6:
            c_r.append(source)
            
    c_r.writeFits(d_directory+"/"+filename)
    return filename.split('-')[0]

In [12]:
r = create_transient_candidate_catalog("catalogs/3sigma",files[0], "tc_catalogs")

In [ ]:
print r

In [15]:
import ipyparallel as ipyp

c = ipyp.Client()
v = c[:]


def f(a,b):
    def c(a, b):
        return a+b

    return c(a,b)
#res = v.map(lambda x,y: x+y, (1, 2, 3, 4), (2, 3, 4, 5))
res = v.map(f, [1, 2, 3, 4], [2, 3, 4, 5])

print res.get()

[3, 5, 7, 9]


In [20]:
directories = ["catalogs/3sigma"]* len(files)
d_directories = ["tc_catalogs/3sigma"]* len(files)
print len(directories)

2098


In [21]:
res = v.map(create_transient_candidate_catalog, directories, files, d_directories)

In [24]:
print res

<AsyncMapResult: create_transient_candidate_catalog:finished>


In [25]:
cats = res.get()
c_cats = list(set(cats))
print len(cats), len(c_cats)

2098 109


In [27]:
import lsst.afw.table as afwTable
import glob

for c_cat in c_cats:
    v_catalog = None
    
    c_files = glob.glob("tc_catalogs/3sigma/"+c_cat+"*.fits")
    for c_file in c_files:
        catalog = afwTable.SourceCatalog.readFits(c_file)
        
        if v_catalog is None:
            v_catalog = catalog.copy()
        else:
            v_catalog.extend(catalog)
    
    v_catalog.writeFits("vc_catalogs/3sigma/"+c_files[0].split("/")[-1].split("-")[0]+".fits")
    

In [30]:
files = os.listdir("vc_catalogs/3sigma")
print files

['852892.fits', '852893.fits', '852891.fits', '839312.fits', '839310.fits', '839311.fits', '853543.fits', '853234.fits', '853541.fits', '853236.fits', '853233.fits', '853232.fits', '853730.fits', '853731.fits', '858540.fits', '858541.fits', '852894.fits', '859609.fits', '860150.fits', '853540.fits', '839309.fits', '839308.fits', '853729.fits', '853539.fits', '853727.fits', '860149.fits', '860148.fits', '860147.fits', '860146.fits', '838868.fits', '838867.fits', '838866.fits', '838865.fits', '838864.fits', '853728.fits', '859608.fits', '844489.fits', '844847.fits', '844845.fits', '844848.fits', '844849.fits', '837004.fits', '837005.fits', '837006.fits', '837007.fits', '837002.fits', '837003.fits', '837008.fits', '851060.fits', '851061.fits', '844498.fits', '844491.fits', '844490.fits', '844493.fits', '844492.fits', '844495.fits', '844494.fits', '844497.fits', '844496.fits', '859612.fits', '850177.fits', '858537.fits', '850179.fits', '850178.fits', '836498.fits', '836493.fits', '836494.f

In [39]:
multi_matches = None
import lsst.afw.geom as afwGeom

for f in files:
    
    catalog = afwTable.SourceCatalog.readFits("vc_catalogs/3sigma/"+f)
    
    if multi_matches is None and len(catalog)>0:
        multi_matches = afwTable.MultiMatch(catalog[0].schema, {'visit':int}, radius=afwGeom.Angle(1./3600., afwGeom.degrees))
    if multi_matches is not None:    
        multi_matches.add(catalog, {'visit':int(f.split(".")[0])})
    


In [41]:
results = multi_matches.finish(removeAmbiguous=False)

In [42]:
def threshold_light_curves(light_curves, threshold):
    t_light_curves = [lc for lc in light_curves if len(lc) >= threshold]
    return t_light_curves


In [43]:
light_curves = []
i = 0
current = -1
while i < len(results):
    result = results[i]
    if current == -1 or current != result['object']:
        lc = [(result['visit'],result)]
        light_curves.append(lc)
        current = result['object']
    else:
        light_curves[-1].append((result['visit'],result))
    i+=1

t_light_curves = threshold_light_curves(light_curves, 10)

In [44]:
len(t_light_curves)

0